In [1]:
import sys
sys.path.append('../') # To import from parent dir

import pandas as pd
from itertools import chain

from util.GrammarBasedUtil import *

In [2]:
# ---- Define the path in which all files are stored ----
folder_path_leno = "../logs/leno/"

sep = ";"
leno_SR_RT_plus = pd.read_csv(folder_path_leno + "2509_extended_SR_RT_plus.csv", sep=sep)
leno_SR_RT_plus_ground_truth = pd.read_csv(folder_path_leno + "2509_extended_SR_RT_plus_ground_truth.csv", sep=sep)
leno_SR_RT_parallel = pd.read_csv(folder_path_leno + "2509_extended_SR_RT_parallel.csv", sep=sep)
leno_SR_RT_parallel_ground_truth = pd.read_csv(folder_path_leno + "2509_extended_SR_RT_parallel_ground_truth.csv", sep=sep)

# ---- Initialize all context parameters that will be used ----
# Note: Read the alignment hints above
systems = []
applications = ["targetApp"]
uiGroup1 = ["url","target.workbookName"]
uiGroup2 = ["target.sheetName"]
uiElement = ["target.id","target.tagName","target.type","target.name","target.href"]
actions = ["eventType"]

# ---- Add all hierarchy levels into a list ----
hierarchy_list_leno = [applications, uiGroup1, uiGroup2, uiElement, actions]
# Flatten the hierarchy list and remove empty lists
hierarchy_columns = list(chain.from_iterable([h for h in hierarchy_list_leno if h]))

Encoding of unique subsequences - Counting their overall occurances in the log

In [17]:
log = symbolize_UILog(leno_SR_RT_plus, hierarchy_columns)
encoding_df, symbols = re_pair(log)
last_entry = len(encoding_df)-1
print(encoding_df[last_entry:])
decoded_symbol = re_pair_decode_symbol("z", encoding_df)

47  unique pairs found so far. Level:  1
63  unique pairs found so far. Level:  2
70  unique pairs found so far. Level:  3
74  unique pairs found so far. Level:  4
        pair new_symbol count
73  (bm, br)         bv     2

Decoding path for symbol 'z':
z → (WJ, WM)
  WJ → [base symbol]
  WM → [base symbol]
Final expansion: ['WJ', 'WM']

